In [1]:
import pandas as pd
import requests
!conda install -c conda-forge folium=0.5.0 --yes #if needed
!conda install -c conda-forge geopy --yes #if needed
import folium
from geopy.geocoders import Nominatim
print('LIBRARIES IMPORTED')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

LIBRARIES IMPORTED


PART 1

In [2]:
#get text from webpage
html=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

#create dataframe with index
df = pd.read_html(html, header = 0)[0]
#ignore cells where Borough is not assigned
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [3]:
#create table joining multiple neighbourhoods
table1 = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#use Borough for Neighbourhoods that are not assigned
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
table1.columns=['Postal Code', 'Borough','Neighborhood']
table1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#determine number of rows and columns 
table1.shape

(103, 3)

PART 2

In [5]:
#get latitude and longitude for each postal code
angles=pd.read_csv('http://cocl.us/Geospatial_data')
angles.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#add latitude and longiture to previous table
table2=pd.merge(table1,angles,on="Postal Code")
table2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


PART 3

In [7]:
#determine latitude and longitute of Toronto
address = 'Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The location of Toronto latitude {} and longitude {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The location of Toronto latitude 43.653963 and longitude -79.387207.


In [8]:
#determine number of boroughs and neighborhoods in Toranto
print('There are {} boroughs and {} neighborhoods in Toranto.'.format(
        len(table2['Borough'].unique()),
        table2.shape[0]))

There are 11 boroughs and 103 neighborhoods in Toranto.


In [9]:
toronto = table2[table2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [10]:
#create map
map1=folium.Map(location=[latitude, longitude])
#add markers to map
for borough, neighborhood, lati, long, in zip(table2['Borough'], table2['Neighborhood'], table2['Latitude'], table2['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label)
    folium.CircleMarker(
        [lati, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map1)
map1 #folium.Map with folium.CircleMarker may not render in Chrome